In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.support.ui import Select

from fuzzywuzzy import fuzz
from time import sleep
import operator

C:\Users\arvind.baranwal\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
def clean(name):
    name = name.lower()
    junk_words = [" corporation"," inc"," ltd"," pvt","."," llc"," llp","&amp",";",","]
    for word in junk_words:
        name = name.replace(word,"")
    name = name.strip()
    return name

In [3]:
def get_browser():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('log-level=3')
    browser = webdriver.Chrome(chrome_options=chrome_options)
    return browser

In [4]:
import pandas as pd
data = pd.read_excel('C:/Users/arvind.baranwal/Downloads/12th july/JanuaryRenewals pulled 7-10-19_CleanedV.01.xlsx',sheetname = 'Cleaned January Renewals')
data.shape

(423, 19)

In [5]:
data.head(2)

,Sr. No.,Identifier,Given Account_Name,Given Street,Given City,Given State,Given Zip,Given Country,LOB,Renewal Date,Input Type,PO BOX,Unique/Duplicate,Cleaned Account_Name,Cleaned Street,Cleaned City,Cleaned State,Cleaned Zip,Cleaned Country
0,830,JAN20REN830,UTILITY CONTRACTORS INC,PO BOX 9592,WICHITA,KS,67277,USA,Auto Lines,2020-01-01,Company,Y,Unique,UTILITY CONTRACTORS INC,PO BOX 9592,WICHITA,KS,67277,USA
1,829,JAN20REN829,INDUSTRIAL SERVICES COMPANY INC,PO BOX 9286,SPRINGFIELD,MO,65803,USA,Auto Lines,2020-01-01,Company,Y,Unique,INDUSTRIAL SERVICES COMPANY INC,PO BOX 9286,SPRINGFIELD,MO,65803,USA


In [6]:
query_company = data['Cleaned Account_Name'].tolist()
original = data['Cleaned Account_Name'].tolist()

In [7]:
len(query_company)

423

In [8]:
# driver= webdriver.Chrome('C:/Users/deepti.patil/Downloads/chromedriver.exe')
driver = get_browser()

url = "https://www.careerbliss.com"

# query_company = ['at&t','cortera','zulily']


company_name =[]
review_link =[]
score_max =[]
original_name=[]

for i in range(0,len(query_company)):
    
#     driver.implicitly_wait(3)
    
    driver.get(url)
    
    #selecting "Companies" tab from the drop down menu
    driver.find_element_by_xpath("""//*[@id="type-dropdown"]/span/b""").click()
    driver.find_element_by_xpath("""//*[@id="type-dropdown"]/ul/li[4]/a""").click()

    co_name = driver.find_element_by_xpath("""//*[@id="search-q"]""")
    co_name.send_keys(query_company[i])

    ##submitting
    driver.find_element_by_xpath("""//*[@id="search-icon"]""").click()
    
    sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    try:

        ### searching for names on the first page only.
        name =[]
        for n in soup.find_all("a",{"class":"large company-name ng-binding"}):
            name.append(n.text.strip())
        
        print(name)

        ## storing list of companies fuzzy score with the query company name
        score=[]
        for s in name:
            score.append(fuzz.ratio(clean(query_company[i]).lower(),clean(s).lower()))
            
        print(score)

        ## cut off as 90-- if more than one cutoff is 90-- choosing max value. What if max value appears more than one time??
        values={}
        for k,v in enumerate(score):
            if v>79:
                values[k] = v

        position = max(values.items(), key=operator.itemgetter(1))[0]
        score_maxvalue = max(values.items(), key=operator.itemgetter(1))[1]
        
        
        ## appending with 
        NAME = name[position]
        ORI_NAME = original[i]
        LINK = "https://www.careerbliss.com"+soup.find_all("a",{"class":"large company-name ng-binding"},href=True)[position]['href']+"reviews/"
        SCORE= score_maxvalue
        
        
    except:
        NAME=""
        ORI_NAME = original[i]
        LINK=""
        SCORE= ""

    company_name.append(NAME)
    review_link.append(LINK)
    score_max.append(SCORE)
    original_name.append(ORI_NAME)
    
driver.quit()
pd.DataFrame({'company_name':company_name,'original_name':original_name,'review_link':review_link,'max_score':score_max}).to_excel('Jan_Renewal_Prospects_careerbliss.xlsx')        

C:\Users\arvind.baranwal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


['ECSM Utility Contractors', 'Utility Data Contractors', 'Altec Industries', 'Solyndra', 'BBSI', 'CEI']
[88, 88, 23, 37, 9, 9]
['Industrial Services', 'Inland Industrial Services Group', 'Arcadia Industrial Services', 'Selway Industrial Services', 'TEAM Industrial Services', 'Quality Industrial Services', 'Williams Industrial Services Group', 'Advanced Industrial Services', 'MPW Industrial Services', 'A & L Industrial Services']
[83, 75, 70, 72, 75, 70, 72, 69, 76, 73]
[]
[]
[]
[]
[]
[]
['Bealls', 'AMC Theatres', 'Daiichi Sankyo', 'Brocade', 'Aggreko', 'Wellpoint', 'Nalco Holding', 'TransPerfect Translations', 'Autoland', 'Bard']
[22, 36, 23, 21, 14, 7, 24, 30, 21, 24]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Allen Tate Co.', 'Allens Inc.', 'The Allen Co.', 'Compuware']
[100, 53, 64, 18]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Walker Die Casting']
[100]
[]
[]
[]
[]
[]
[]
['Pioneer Metal Finishing', 'Amistar']
[100, 20]
[]
[]
['ATA Airlines', 'Cardinal Logistics Manag

['Rheem', 'Novo Nordisk', 'McKesson']
[18, 25, 28]
[]
[]
[]
[]
['Blue Cross Blue Shield', 'Wellpoint']
[81, 20]
[]
[]
['Brand Services', 'Yoh', 'Hughes Supply']
[39, 4, 15]
['STG Inc.', 'TJX Companies', 'The Inland Real Estate Group of Companies', 'Lindsey Management', 'STONI Management', 'Seabreeze Management Company', 'Rudin Management Co.', 'Premier Restaurant Management Co.', 'Progressive Employer Management Co.', 'Integrated Project Management Co.']
[7, 41, 42, 59, 62, 74, 71, 59, 53, 59]
['Boyle Hotels']
[52]
[]
[]
[]
[]
[]
[]
[]
[]
['Career Education', 'Boeing', 'Metropolitan State College of Denver', 'New York Institute of Technology', 'QAD', 'University of Detroit Mercy']
[35, 17, 15, 24, 10, 27]
['Valu Home Centers', 'Albertsons']
[100, 37]
[]
[]
[]
[]
[]
[]
['Mesa Home Products', 'Ceradyne', 'EnGenius', 'Pacific Dental Services', "Anna's Linens", 'TTM Technologies', 'Adir International']
[84, 29, 19, 11, 15, 21, 19]
['Indeck Power Equipment Co.', 'Reagan Equipment', 'Hitachi

[]
[]
[]
[]
[]
[]
[]
[]
['Alpha Construction Co., Inc.', 'Kokoska Construction Co.', 'Superior Construction', 'DK Construction Co.', 'Beador Construction Co.', 'Riverside Construction Co.', 'Cash Construction Co.', 'Chanen Construction Co.', 'Elite Construction Co.', 'Blake Construction Co.']
[76, 72, 93, 76, 83, 78, 77, 74, 80, 76]
['Harrington Industrial Plastics']
[100]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['National Jewish Health', 'Cedars Sinai Medical Center']
[100, 33]
['Parsons', 'Parsons Brinckerhoff', 'WorleyParsons', 'Parsons and Company Custom Homes', 'Intecsea Worley Parsons Group', 'GoDaddy.com']
[64, 57, 50, 64, 45, 32]
['Paula LeDuc Fine Catering']
[81]
[]
[]
[]
[]
['AFE International Group', 'Insight Global']
[23, 12]
[]
[]
['Iron Mountain Digital']
[10]
[]
[]
['Red Ventures']
[40]
[]
[]
[]
[]
['Trulite Glass & Aluminum Solutions']
[74]
['Right Management', 'RD Management', 'Megaversal Management', 'Boyd Management', 'Oakcrest Management', 'Epoch Management', 'Cas

In [16]:
import pandas as pd
pd.DataFrame({'company_name':company_name,'original_name':original_name,'review_link':review_link,'max_score':score_max}).to_excel('C:/Users/arvind.baranwal/Downloads/1 july/Nov effect prospects_careerbliss.xlsx')

## built condensed function for getting all matches

In [5]:


def get_browser():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('log-level=3')
    browser = webdriver.Chrome(chrome_options=chrome_options)
    return browser

def clean(name):
    name = name.lower()
    junk_words = [" corporation"," inc"," ltd"," pvt","."," llc"," llp","&amp",";",","]
    for word in junk_words:
        name = name.replace(word,"")
    name = name.strip()
    return name

driver = get_browser()

def candidates_careerbliss_1(companyname,match_cutoff):
    
    driver = get_browser()
    url = "https://www.careerbliss.com"

    driver.get(url)
    driver.find_element_by_xpath("""//*[@id="type-dropdown"]/span/b""").click()
    driver.find_element_by_xpath("""//*[@id="type-dropdown"]/ul/li[4]/a""").click()

    co_name = driver.find_element_by_xpath("""//*[@id="search-q"]""")
    co_name.send_keys(companyname.lower())

    ##submitting
    driver.find_element_by_xpath("""//*[@id="search-icon"]""").click()
    
    sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    try:

        ### searching for names on the first page only.
        name =[]
        for n in soup.find_all("a",{"class":"large company-name ng-binding"}):
            name.append(n.text.strip())

        ## storing list of companies fuzzy score with the query company name
        scores=[]
        found_names = []
        review_links = []
        position = 0
        for s in name:
            match_score = fuzz.ratio(clean(companyname).lower(),clean(s).lower()) 
            if match_score > match_cutoff:
                found_names.append(s)
                review_links.append("https://www.careerbliss.com"+soup.find_all("a",{"class":"large company-name ng-binding"},href=True)[position]['href']+"reviews/")
                scores.append(match_score)
            position+=1             
        #         print('names after matching cut-off')
        #         print(len(found_names))
        #         print(found_names)
        #         print('review links')
        #         print(len(review_links))
        #         print(review_links)
        #         print('matching scores')
        #         print(len(scores))
        #         print(scores)

    except:
        found_names = []
        review_links = []
        scores=[]  
    
    driver.refresh 
    return found_names, review_links, scores


C:\Users\arvind.baranwal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [7]:
candidates_careerbliss_1('Minnesota Housing Finance Agency',80)

C:\Users\arvind.baranwal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


(['Minnesota Housing Finance Agency', 'DC Housing Finance Agency'],
 ['https://www.careerbliss.com/minnesota-housing-finance-agency/reviews/',
  'https://www.careerbliss.com/dchfa/reviews/'],
 [100, 81])